In [5]:
import json
import os
import tarfile
import warnings

import numpy as np
import pandas as pd
import requests
import spacy
# import wget
# from elevate import elevate
from spacy.lang.en import English

import _pickle as pickle
from flask import Flask
# from flask_cors import CORS

import pandas as pd
from collections import Counter
import re
import spacy
from spacy.tokenizer import Tokenizer
from nltk.stem import PorterStemmer
import scattertext as st
from pprint import pprint

#Imports
from json import loads
from lxml import html
from requests import Session
import pandas as pd
from concurrent.futures import ThreadPoolExecutor as Executor
from itertools import count

In [6]:
warnings.filterwarnings('ignore')

df = pd.read_csv('C:\\Users\\lilyx\\BR-Data-Science\\test-version-v25\\trimmed_20k.csv')

lg_url = "https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.1.0/en_core_web_sm-2.1.0.tar.gz"

r = requests.get(lg_url, allow_redirects=True)
open('sm.zip', 'wb').write(r.content)

tar = tarfile.open('sm.zip', "r:gz")
tar.extractall('down_sm')

nlp = spacy.load("./down_sm/en_core_web_sm-2.1.0/en_core_web_sm/en_core_web_sm-2.1.0")
nlp.Defaults.stop_words |= {'check-in','=','= =','male','u','want', 'u want', 'cuz','him',"i've", 'deaf','on', 'her','told','told him','ins', '1 check','I', 'i"m', 'i', ' ', 'it', "it's", 'it.','they','coffee','place','they', 'the', 'this','its', 'l','-','they','this','don"t','the ', ' the', 'it', 'i"ve', 'i"m', '!', '1','2','3','4', '5','6','7','8','9','0','/','.',','}


In [14]:
def get_termfreq_from_url(yelp_url, from_isbn=False):
    '''Takes a url, scrape site for reviews
    and calculates the term frequencies 
    sorts and returns the top 10 as a json object
    containing term, highratingscore, poorratingscore.'''
    
    base_url = "https://www.yelp.com/biz/" # add business id
    api_url = "/review_feed?sort_by=date_desc&start="
    bid = url[24:] # business id

    class Scraper():
        def __init__(self):
            self.data = pd.DataFrame()

        def get_data(self, n, bid=bid):
            with Session() as s:
                with s.get(base_url+bid+api_url+str(n*20)) as resp: #makes an http get request to given url and returns response as json
                    r = loads(resp.content) #converts json response into a dictionary
                    _html = html.fromstring(r['review_list']) #loads from dictionary

                    dates = _html.xpath("//div[@class='review-content']/descendant::span[@class='rating-qualifier']/text()")
                    reviews = [el.text for el in _html.xpath("//div[@class='review-content']/p")]
                    ratings = _html.xpath("//div[@class='review-content']/descendant::div[@class='biz-rating__stars']/div/@title")

                    df = pd.DataFrame([dates, reviews, ratings]).T

                    self.data = pd.concat([self.data,df])

        def scrape(self): #makes it faster
            # multithreaded looping
            with Executor(max_workers=40) as e:
                list(e.map(self.get_data, range(10)))

    s = Scraper()
    s.scrape()
    df = s.data
    df = df.sample(100)

    corpus = st.CorpusFromPandas(df, 
                             category_col=2, 
                             text_col=1,
                             nlp=nlp).build()

    term_freq_df = corpus.get_term_freq_df()
    term_freq_df['highratingscore'] = corpus.get_scaled_f_scores('5.0 star rating')

    term_freq_df['poorratingscore'] = corpus.get_scaled_f_scores('1.0 star rating')

    df = term_freq_df.sort_values(by= 'poorratingscore', ascending = False)

    df['highratingscore'] = round(df['highratingscore'], 2)
    df['poorratingscore'] = round(df['poorratingscore'], 2)
    
    df.to_json(r'test.json', orient='table')
#     #scrape
    
#     #instantiate empty list of rating scores:
#     ratings = []
    
#     #iterate over the doc object for each book in the df to get the similarity score and append to list
#     for doc in df.docs:
#         sim = eachterm.generateRatingScores(doc)#returns scores
#         ratings.append(sim)
        
#     #sort the list and grab the top 10:
#     if from_isbn:
#         #skip the 0th ranked book which will be the bookused to get the yelp_url of the description:
#         top10 = pd.Series(sims).sort_values(ascending=False).iloc[1:11]
#     else:
#         top10 = pd.Series(sims).sort_values(ascending=False).iloc[:10]

#     #instantiate empty list to store the python dicts of each book
#     term = []

#     #iterate thru the top 10 ranked simlilar books and populate the book list w/ dictionaries for each book

#     for i in top10.index:
#         term = {}
#     #     term['word'] = df.iloc[i]['word']
#         term['highratingscore'] = df.iloc[i]['highratingscore']
#         term['poorratingscore'] = str(df.iloc[i]['poorratingscore'])
#         term.append(term)
#     return json.dumps(term)

In [15]:
yelp_url = 'https://www.yelp.com/biz/lobster-house-all-you-can-eat-seafood-rego-park-30'
get_termfreq_from_url(yelp_url, from_isbn=False)

In [16]:
pickle.dump(get_termfreq_from_url, open('termFr01.pkl', 'wb'))

In [20]:
data = json.dumps({'yelp_url': 'https://www.yelp.com/biz/lobster-house-all-you-can-eat-seafood-rego-park-30'})

url = 'http://localhost:9000/api'

send = requests.post(url, data)

print(send.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import json
import os
import tarfile
import warnings

import numpy as np
import pandas as pd
import requests
import spacy
import wget

import _pickle as pickle

import pandas as pd
from collections import Counter
import re
import spacy
import scattertext as st
from pprint import pprint

#Imports
from json import loads
from lxml import html
from requests import Session
import pandas as pd
from itertools import count
import os
from flask import Flask, render_template, request
from flask_cors import CORS

#we are not using jupyter notebook, so no jsonify

#pred logic, grabs information for us from post
def ValuePredictor(yelp_url):
    '''Takes a url, scrape site for reviews
    and calculates the term frequencies 
    sorts and returns the top 10 as a json object
    containing term, highratingscore, poorratingscore.'''
    
    base_url = "https://www.yelp.com/biz/" # add business id
    api_url = "/review_feed?sort_by=date_desc&start="
    bid = yelp_url.replace('https://www.yelp.com/biz/','')

    class Scraper():
        def __init__(self):
            self.data = pd.DataFrame()

        def get_data(self, n, bid=bid):
            with Session() as s:
                with s.get(base_url+bid+api_url+str(n*20)) as resp: #makes an http get request to given url and returns response as json
                    r = resp.json() #converts json response into a dictionary
                    _html = html.fromstring(r['review_list']) #loads from dictionary

                    dates = _html.xpath("//div[@class='review-content']/descendant::span[@class='rating-qualifier']/text()")
                    reviews = [el.text for el in _html.xpath("//div[@class='review-content']/p")]
                    ratings = _html.xpath("//div[@class='review-content']/descendant::div[@class='biz-rating__stars']/div/@title")

                    df = pd.DataFrame([dates, reviews, ratings]).T

                    self.data = pd.concat([self.data,df])

        def scrape(self): #makes it faster
            # multithreaded looping
            with Executor(max_workers=40) as e:
                list(e.map(self.get_data, range(10)))

    s = Scraper()
    s.scrape()
    df = s.data
    df = df.sample(100)

    corpus = st.CorpusFromPandas(df, 
                             category_col=2, 
                             text_col=1,
                             nlp=nlp).build()

    term_freq_df = corpus.get_term_freq_df()
    term_freq_df['highratingscore'] = corpus.get_scaled_f_scores('5.0 star rating')

    term_freq_df['poorratingscore'] = corpus.get_scaled_f_scores('1.0 star rating')

    df = term_freq_df.sort_values(by= 'poorratingscore', ascending = False)

    df['highratingscore'] = round(df['highratingscore'], 2)
    df['poorratingscore'] = round(df['poorratingscore'], 2)
    
    df.to_json(r'test.json', orient='table')#similar to the logic of make_predict function
#all the above does the prediction for you

#app
app=Flask(__name__)
CORS(app)

#routes
@app.route('/')#defaults to this just in case, legacy reasons
@app.route('/index')
def index():
    return flask.render_template('index.html')#we are going to have a form

#we have to have something to hold and run the results page
@app.route('/result', methods = ['POST'])
def result():#will capture our predictions, handles result
#result will grab post from index.html, instead of host
    if request.method == 'POST':
        content_type = request.headers["content-type"]
        if content_type == "application/json":
        #take information from form with nominal data, let html do the work convert to dictionary
            to_predict_list = request.form.to_dict()
            to_predict_list = list(to_predict_list.values())
            to_predict_list = list(map(str, to_predict_list))
            result = ValuePredictor(to_predict_list)
        return render_template("results.html", prediction=result)
    else:
        return jsonify("Content-Type is not application/json")
#app run, so we don't export as an actual app
if __name__ == '__main__':
    app.run(port=9000, debug=True)
#windows go to system variables to shut down.

In [11]:
import json
import os
import tarfile
import warnings

import numpy as np
import pandas as pd
import requests
import spacy
# import wget

import _pickle as pickle

import pandas as pd
from collections import Counter
import re
import spacy
import scattertext as st
from pprint import pprint

#Imports
from json import loads
from lxml import html
from requests import Session
import pandas as pd
from itertools import count
import os

from concurrent.futures import ThreadPoolExecutor as Executor
# from flask import Flask, render_template, request
# from flask_cors import CORS

# #we are not using jupyter notebook, so no jsonify

# #pred logic, grabs information for us from post
# def ValuePredictor(yelp_url):
#     '''Takes a url, scrape site for reviews
#     and calculates the term frequencies 
#     sorts and returns the top 10 as a json object
#     containing term, highratingscore, poorratingscore.'''
# yelp_url = 'https://www.yelp.com/biz/dig-inn-new-york-14'
base_url = "https://www.yelp.com/biz/" # add business id
api_url = "/review_feed?sort_by=date_desc&start="
bid = 'dig-inn-new-york-14'

class Scraper():
    def __init__(self):
        self.data = pd.DataFrame()

    def get_data(self, n, bid=bid):
        with Session() as s:
            with s.get(base_url+bid+api_url+str(n*20)) as resp: #makes an http get request to given url and returns response as json
                r = resp.json() #converts json response into a dictionary
                _html = html.fromstring(r['review_list']) #loads from dictionary

                dates = _html.xpath("//div[@class='review-content']/descendant::span[@class='rating-qualifier']/text()")
                reviews = [el.text for el in _html.xpath("//div[@class='review-content']/p")]
                ratings = _html.xpath("//div[@class='review-content']/descendant::div[@class='biz-rating__stars']/div/@title")

                df = pd.DataFrame([dates, reviews, ratings]).T

                self.data = pd.concat([self.data,df])

    def scrape(self): #makes it faster
        # multithreaded looping
        with Executor(max_workers=40) as e:
            list(e.map(self.get_data, range(10)))

s = Scraper()
s.scrape()
df = s.data
df.shape

(213, 3)

In [12]:
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

C:\Users\lilyx\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [62]:
df.head(2)

,0,1,2
0,\n 10/27/2015\n,"If, like me, you've recently realized you can'...",5.0 star rating
1,\n 10/19/2015\n,I give this place 3 stars because I like the c...,3.0 star rating


In [64]:
df.dropna(inplace = True)
documents = df[1]

STOPWORDS = set(STOPWORDS).union(set(['said', 'mr', 'mrs','check-in','=','= =','male','u','want', 'u want', 'cuz','him',"i've", 'deaf','on', 'her','told','told him','ins', '1 check','I', 'i"m', 'i', ' ', 'it', "it's", 'it.','they','coffee','place','they', 'the', 'this','its', 'l','-','they','this','don"t','the ', ' the', 'it', 'i"ve', 'i"m', '!', '1','2','3','4', '5','6','7','8','9','0','/','.',',']))


texts = [
    [word for word in document.lower().split() if word not in STOPWORDS]
    for document in documents]

In [69]:
id2word = corpora.Dictionary(texts)
id2word.token2id['realized']

9

In [74]:
# id2word.token2id

import sys
# sys.getsizeof(id2word)
len(id2word.keys())

# Let's remove extreme values from the dataset
id2word.filter_extremes(no_below=3, no_above=0.97)#just like max_df
#word has to appear in at least 10 documents to be included. 
#75% of documents have the word in it
#at least 10 documents less than 75% documents
len(id2word.keys())

405

In [75]:
corpus = [id2word.doc2bow(text) for text in texts]#storing only vector representations

lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,#for reproducability not very stable, different results
                   num_topics = 25,#number of topics
                   passes=10,
                   workers=4#multi-core
                  )

In [76]:
lda.print_topics()

[(0,
  '0.063*"sweet" + 0.044*"potatoes" + 0.041*"mac" + 0.032*"got" + 0.029*"healthy" + 0.029*"brown" + 0.026*"pretty" + 0.025*"good." + 0.021*"good" + 0.021*"cheese"'),
 (22,
  '0.039*"food" + 0.029*"like" + 0.024*"good" + 0.024*"don\'t" + 0.019*"options" + 0.019*"know" + 0.019*"dig" + 0.019*"big" + 0.015*"fan" + 0.015*"great"'),
 (7,
  '0.047*"food" + 0.045*"dig" + 0.023*"staff" + 0.023*"food," + 0.023*"looking" + 0.023*"you\'re" + 0.023*"mac" + 0.023*"cheese" + 0.023*"inn" + 0.023*"day."'),
 (13,
  '0.030*"food" + 0.025*"best" + 0.025*"quality" + 0.025*"sure" + 0.017*"friendly" + 0.017*"inn" + 0.017*"great" + 0.017*"employees" + 0.017*"sweet" + 0.017*"dig"'),
 (10,
  '0.066*"great" + 0.036*"mac" + 0.030*"cheese" + 0.024*"food" + 0.024*"especially" + 0.018*"quick" + 0.018*"dig" + 0.018*"staff" + 0.018*"service" + 0.012*"spot"'),
 (21,
  '0.054*"food" + 0.032*"way" + 0.022*"quick" + 0.022*"nice" + 0.022*"healthy" + 0.022*"meal." + 0.022*"great" + 0.022*"price" + 0.022*"quality" + 0.0

In [77]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [78]:
topics = [' '.join(t[0:5]) for t in words]

In [79]:
for t in topics:
    print(t)
    print('\n')

food great dig fresh time


healthy good great like food,


food way quick nice healthy


food sides inn sweet dig


great dig healthy chicken inn


food best quality sure friendly


healthy food lunch looking quick,


great food dig selection service


great places use fast food.


food dig staff food, looking


food come love healthy inn


inn choices dig far great


like food lunch love great


delicious! good! you're isn't --


sweet potatoes mac got healthy


food like good don't options


staff great foods healthy especially


eat salad chicken dig came


food got time definitely changed


food good great quick roasted




In [81]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\lilyx\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
24     0.107890 -0.050365       1        1  6.659113
22    -0.050958  0.008751       2        1  6.485621
1      0.097050 -0.003350       3        1  6.346934
14     0.019819 -0.031218       4        1  5.789328
5      0.003907 -0.034399       5        1  5.746469
6      0.035366 -0.037040       6        1  5.563725
10     0.080966 -0.010517       7        1  5.071084
0      0.041207  0.178413       8        1  4.826768
12     0.000059 -0.111386       9        1  4.816307
20    -0.140053  0.113417      10        1  4.798189
23    -0.084171 -0.120398      11        1  4.388112
9      0.001868 -0.037914      12        1  4.070121
17     0.012281  0.014411      13        1  4.028997
4     -0.090158  0.022330      14        1  3.978689
13     0.085745  0.104017      15        1  3.697099
16     0.040609  0.033738      16        1  3.553245
15     0.051762  0.064596      17        1  3.413165
19    -0.139059  0.048363      18        1  2.768389
21     0.013917 -0.101842      19        1  2.681504
7      0.076418 -0.035323      20        1  2.526950
8     -0.079664 -0.118069      21        1  2.436089
11     0.032650  0.014318      22        1  2.359825
3     -0.141520  0.030188      23        1  2.322104
2      0.065323  0.032704      24        1  1.427871
18    -0.041254  0.026576      25        1  0.244300, topic_info=    Category       Freq        Term      Total  loglift  logprob
64   Default  63.000000       great  63.000000  30.0000  30.0000
62   Default  98.000000        food  98.000000  29.0000  29.0000
1    Default  49.000000         dig  49.000000  28.0000  28.0000
103  Default  27.000000       sweet  27.000000  27.0000  27.0000
59   Default  43.000000     healthy  43.000000  26.0000  26.0000
57   Default  28.000000       staff  28.000000  25.0000  25.0000
4    Default  34.000000        like  34.000000  24.0000  24.0000
213  Default  27.000000         mac  27.000000  23.0000  23.0000
144  Default  28.000000        love  28.000000  22.0000  22.0000
141  Default  18.000000  definitely  18.000000  21.0000  21.0000
78   Default  22.000000       lunch  22.000000  20.0000  20.0000
21   Default  28.000000        time  28.000000  19.0000  19.0000
105  Default  23.000000         got  23.000000  18.0000  18.0000
314  Default  15.000000      cheese  15.000000  17.0000  17.0000
13   Default  35.000000         inn  35.000000  16.0000  16.0000
90   Default  16.000000       sides  16.000000  15.0000  15.0000
130  Default  10.000000       salad  10.000000  14.0000  14.0000
316  Default  10.000000    potatoes  10.000000  13.0000  13.0000
51   Default  16.000000       food,  16.000000  12.0000  12.0000
38   Default  29.000000        good  29.000000  11.0000  11.0000
2    Default  11.000000         eat  11.000000  10.0000  10.0000
115  Default  15.000000       brown  15.000000   9.0000   9.0000
137  Default   9.000000         way   9.000000   8.0000   8.0000
47   Default  12.000000     looking  12.000000   7.0000   7.0000
67   Default  21.000000     chicken  21.000000   6.0000   6.0000
10   Default   8.000000         far   8.000000   5.0000   5.0000
117  Default  12.000000        come  12.000000   4.0000   4.0000
202  Default  17.000000       quick  17.000000   3.0000   3.0000
32   Default  15.000000        fast  15.000000   2.0000   2.0000
183  Default  12.000000  especially  12.000000   1.0000   1.0000
..       ...        ...         ...        ...      ...      ...
185  Topic25   0.017644        mint   3.470893   0.7328  -6.0039
149  Topic25   0.017644      ginger   3.470977   0.7327  -6.0039
404  Topic25   0.017644     cooked.   3.519908   0.7187  -6.0039
137  Topic25   0.017644         way   9.445094  -0.2683  -6.0039
128  Topic25   0.017644      pretty  16.654968  -0.8355  -6.0039
129  Topic25   0.017644    register   3.480775   0.7299  -6.0039
130  Topic25   0.017644       salad  10.370597  -0.3618 

In [95]:
#analysis over document topic distributions
lda[corpus[0]]#pass in whatever documents in lda, first chapter in our splits
distro = [lda[d] for d in corpus]#get distribution of documents in corpus
distro[0]#view probabilities
def update(doc):#transform a distro in a complete distribution including zeros
        d_dist = {k:0 for k in range(0,24)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]
new_distro
d_dist = {k:0 for k in range(0,24)}
df = pd.DataFrame.from_records(new_distro, index=1)
# df.columns = topics
# df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()
df.head()

KeyError: 'index'

In [87]:
len(topics)

20